##### ЗАДАЧИ ДЛЯ ПРАКТИКИ ООП

**1. БАЗОВЫЕ ФИНАНСОВЫЕ ИНСТРУМЕНТЫ:**
- Реализуйте класс `EuropeanPutOption`, наследующий от `Option`
- Реализуйте класс `AmericanCallOption` с возможностью досрочного исполнения
- Добавьте валидацию даты истечения в базовый класс `Option`
- Создайте класс `Stock` (акция) как наследник `AbstractAsset`

In [ ]:
from abc import ABC, abstractmethod
import math
from datetime import datetime
import time
from scipy.stats import norm

###         Inheritance strategy
# AbstractAsset -> Derivative -> Option -> European
# AbstractAsset -> Stock

class AbstractAsset(ABC):
    """
        Abstract class for all assets

        Args:
            name (str): name of the asset
            price (float): price of the asset
    """
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = price

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}: {self.name}. Price: {self.price}'


class Stock(AbstractAsset):
    """
        Class for stock assets (basic asset)
        Global registry for all stocks (singleton pattern), to ensure, that Stock 'name' is a single object.

        Args:
            name (str): name of the stock
            price (float): price of the stock

        Class attributes:
            __registry (dict): registry for all stocks
    """

    __registry: dict[str, 'Stock'] = dict()

    def __new__(cls, *args, **kwargs):
        # args[0] — это name, если передан позиционно
        if 'name' in kwargs:
            name = kwargs['name']
        elif len(args) >= 1:
            name = args[0]
        else:
            raise TypeError("Stock() missing required argument: 'name'")
        
        if name in cls.__registry:
            return cls.__registry[name]
        
        instance = super().__new__(cls)
        cls.__registry[name] = instance
        return instance

    def __init__(self, name: str, price: float):
        # Initialize stock object only if it's not initialized yet
        if not hasattr(self, 'name'):
            super().__init__(name, price)

    @classmethod # attached to class, not to instance
    def get_from_registry(cls, name: str) -> 'Stock':
        # Getter (class method) for stock object from registry. Returns stock object if it exists, otherwise raises KeyError.
        if name not in cls.__registry: 
            raise KeyError(f"Stock {name} doesn't exist")
        return cls.__registry[name]
    
    
class Derivative(AbstractAsset):
    """
        Base class for derivative financial instruments (options, futures, etc.)

        Args:
            name (str): name of the derivative
            price (float): current market price of the derivative
            expiration_date (str): expiration date in 'YYYY-MM-DD' format
            underlying_name (str): name of the underlying asset

        Class attributes:
            FORMAT_CODE (str): expected date format for expiration_date
    """

    FORMAT_CODE = "%Y-%m-%d"


    def __init__(self, name: str, price: float, expiration_date: str, underlying_name: str):
        super().__init__(name, price)
        # Parse and validate expiration date
        self.expiration_date = datetime.strptime(expiration_date, self.FORMAT_CODE)
        # Link to underlying asset from registry
        self.__underlying: Stock = Stock.get_from_registry(underlying_name)
    
    @property
    def underlying(self) -> Stock:
        # read-only access to the underlying asset
        return self.__underlying

    @property
    def time_to_maturity(self) -> float:
        """Time to maturity in years (expiration_date - now / 365)."""
        now = datetime.now()
        if self.expiration_date < now:
            return 0.0
        delta = self.expiration_date - now
        return delta.days / 365.0

class Option(Derivative):
    """
        Abstract class for options (inherited from Derivative)

        Args:
            name (str)
            price (float)
            expiration_date (str)
            underlying_name (str)
            strike_price (float)
            risk_free_rate (float)
            volatility (float)
    """
    def __init__(self, name: str, price: float, expiration_date: str, underlying_name: str,
                 strike_price: float, risk_free_rate: float, volatility: float):
        super().__init__(name, price, expiration_date, underlying_name)
        self.strike_price = strike_price
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility

    @abstractmethod
    def exercise(self) -> float:
        """Payoff at exercise"""
        pass

    @abstractmethod
    def price(self) -> float:
        """Theoretical option price"""
        pass

    # Greek methods (not implemented and yet not abstract methods)
    def delta(self) -> float:
        pass
    def gamma(self) -> float:
        pass
    def theta(self) -> float:
        pass
    def vega(self) -> float:
        pass


class EuropeanPutOption(Option):
    """
        European put option: can be exercised only at expiration

        Payoff: max(strike_price - underlying_price, 0)
    """

    def price(self) -> float:
        # Black-Scholes price for European put option
        T = self.time_to_maturity
        if T <= 0:
            return self.exercise()
        S = self.underlying.price
        K = self.strike_price
        r = self.risk_free_rate
        sigma = self.volatility

        d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
        d2 = d1 - sigma * math.sqrt(T)
        put_price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return put_price
        
    def exercise(self) -> float:
        # European option cannot be exercised before expiration
        if self.time_to_maturity > 0:
            raise ValueError("European option can only be exercised at expiration")
        # Returns payoff after exercise
        return max(self.underlying.price - self.strike_price, 0)


class AmericanCallOption(Option):
    """
        American call option class. It can be exercised at any time until expiration

        Payoff: max(underlying_price - strike_price, 0)
    """

    def exercise(self) -> float:
        # American options can be exercised at any time. Return payoff
        return max(self.underlying.price - self.strike_price, 0)

    def price(self) -> float:
        T = self.time_to_maturity
        if T <= 0:
            return self.exercise()
        S = self.underlying.price
        K = self.strike_price
        r = self.risk_free_rate
        sigma = self.volatility

        d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
        d2 = d1 - sigma * math.sqrt(T)
        call_price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
        return call_price

In [3]:
sber: Stock = Stock(name = 'SBER', price = 300)
Stock.get_from_registry('SBER')

Stock: SBER. Price: 300

In [4]:
vtbr = Stock(name = 'VTBR', price = 70)

eur_put_option = EuropeanPutOption(
    name = 'Default Name', 
    strike_price = 70,
    risk_free_rate = 0.18,
    volatility = 0.3,
    expiration_date = '2025-12-31',
    price = 7,
    underlying_name = 'VTBR'
)

am_call_option = AmericanCallOption(
    name = 'Default Name', 
    strike_price = 70,
    risk_free_rate = 0.18,
    volatility = 0.3,
    expiration_date = '2025-12-31',
    price = 7,
    underlying_name = 'VTBR'
)

In [5]:
print(am_call_option.exercise())

try:
    eur_put_option.exercise()
except ValueError as e:
    print(f"Raised ValueError: {e}")

0
Raised ValueError: European option can only be exercised at expiration


**2. РАСШИРЕННЫЕ ОПЦИОНЫ:**
- Реализуйте класс `BinaryOption` (бинарный опцион)
- Создайте класс `BarrierOption` (барьерный опцион)
- Добавьте метод расчета Greeks (дельта, гамма, тета, вега) для опционов

In [6]:
# TODO
# 1) реализовать BinaryOption(fixed_payoff)
# 2) реализовать BarrierOption(P>0 if S_t in [B_L, B_U])
# 3) Greeks сделать без рассчетов (pass), "этот метод существует"

In [ ]:
class BinaryOption(Option):
    """
        Binary (digital) option: pays fixed amount if condition is met at expiration

        Payoff: 1 if underlying_price > strike_price, else 0
    """

    def exercise(self) -> float:
        # Binary option pays 1 if in-the-money at expiration, else 0
        if self.time_to_maturity > 0:
            raise ValueError("Binary option can only be exercised at expiration")
        return 1.0 if self.underlying.price > self.strike_price else 0.0

    def price(self) -> float:
        # Theoretical price of cash-or-nothing binary call (Black-Scholes)
        T = self.time_to_maturity
        if T <= 0:
            return self.exercise()
        S = self.underlying.price
        K = self.strike_price
        r = self.risk_free_rate
        sigma = self.volatility

        d2 = (math.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
        binary_price = math.exp(-r * T) * norm.cdf(d2)
        return binary_price



class BarrierOption(Option):
    """
        Barrier option: becomes active or inactive when underlying hits a barrier level
        
        Barrier Option behaviour:
        - worthless if underlying price >= barrier
        - standard European option
    """

    def __init__(self, name: str, price: float, expiration_date: str, underlying_name: str,
                 strike_price: float, risk_free_rate: float, volatility: float, barrier: float):
        super().__init__(name, price, expiration_date, underlying_name, strike_price, risk_free_rate, volatility)
        self.barrier = barrier

    def exercise(self) -> float:
        if self.time_to_maturity > 0:
            raise ValueError("Barrier option can only be exercised at expiration")

        if self.underlying.price >= self.barrier:
            return 0.0  # option is disabled (hit the barrier)
        return max(self.underlying.price - self.strike_price, 0)

    def price(self) -> float:
        # Return 0 if already knocked out, otherwise European call price
        if self.underlying.price >= self.barrier:
            return 0.0
        T = self.time_to_maturity
        if T <= 0:
            return self.exercise()
        
        # European call pricing
        S = self.underlying.price
        K = self.strike_price
        r = self.risk_free_rate
        sigma = self.volatility

        d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
        d2 = d1 - sigma * math.sqrt(T)
        call_price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
        return call_price

In [8]:
from datetime import timedelta

aapl = Stock(name="AAPL", price=190.0)
exp_date = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d") # tomorrow date

barrier_opt = BarrierOption(
    name="AAPL_BARRIER",
    price=0.0,
    expiration_date=exp_date,
    underlying_name="AAPL",
    strike_price=185.0,
    risk_free_rate=0.05,
    volatility=0.25,
    barrier=200.0  # если AAPL достигнет 200 — опцион аннулируется
)

barrier_opt.price

0.0

**3. ПОРТФЕЛЬ И УПРАВЛЕНИЕ АКТИВАМИ:**
- Реализуйте метод `__repr__` для класса `Portfolio`
- Добавьте метод `__getitem__` для доступа к активам по индексу
- Реализуйте метод `__contains__` для проверки наличия актива в портфеле
- Добавьте метод `remove_asset` для удаления актива из портфеля
- Создайте метод `calculate_portfolio_risk` для расчета риска портфеля

**5. ДОПОЛНИТЕЛЬНЫЕ DUNDER МЕТОДЫ:**
- Добавьте `__eq__`, `__lt__`, `__le__` для сравнения портфелей
- Реализуйте `__mul__` для умножения портфеля на число (масштабирование)
- Добавьте `__str__` для красивого вывода портфеля
- Реализуйте `__bool__` для проверки, является ли портфель пустым

In [ ]:
class Portfolio:
    """
        Class for managing a collection of financial assets (stocks, options, etc.)

        Assets are stored in a dictionary keyed by asset name.
        If an asset with the same name is added, it replaces the existing one.

        Args:
            assets (list[AbstractAsset], optional): initial list of assets. Defaults to empty list.

        Notes:
            - All comparison operations are based on total portfolio value (sum of asset prices).
            - Asset names must be unique within the portfolio.
    """

    def __init__(self, assets=None):
        # Store assets as {name: asset}
        self._assets = {}
        if assets:
            for asset in assets:
                self._assets[asset.name] = asset

    def __repr__(self) -> str:
        # Developer-friendly representation
        return f'Portfolio(assets={len(self._assets)})'

    def __str__(self) -> str:
        # User-friendly string with list of all assets
        if not self._assets:
            return "Portfolio: (empty)"
        asset_lines = [f"  {name}: {asset}" for name, asset in self._assets.items()]
        return "Portfolio:\n" + "\n".join(asset_lines)

    def __getitem__(self, name: str) -> AbstractAsset:
        # Access asset by name: portfolio['AAPL']
        return self._assets[name]

    def __contains__(self, name: str) -> bool:
        # Check if asset name is in portfolio: 'AAPL' in portfolio
        return name in self._assets

    def remove_asset(self, name: str) -> None:
        # Remove asset by name; raises KeyError if not found
        if name not in self._assets:
            raise KeyError(f"Asset '{name}' not found in portfolio")
        del self._assets[name]

    def calculate_portfolio_risk(self):
        pass

    def __bool__(self) -> bool:
        # Return False if portfolio is empty
        return len(self._assets) > 0

    def __mul__(self, factor: float):
        # Scale portfolio by a number: new_portfolio = portfolio * 2.0
        if not isinstance(factor, (int, float)):
            return NotImplemented
        scaled_assets = []
        for asset in self._assets.values():
            scaled_asset = asset.__class__(name=asset.name, price=asset.price * factor)
            scaled_assets.append(scaled_asset)
        return Portfolio(scaled_assets)

    def __rmul__(self, factor: float):
        return self.__mul__(factor)

    def _total_value(self) -> float:
        return sum(asset.price for asset in self._assets.values())

    def __eq__(self, other) -> bool:
        if not isinstance(other, Portfolio):
            return NotImplemented
        return self._total_value() == other._total_value()

    def __lt__(self, other) -> bool:
        if not isinstance(other, Portfolio):
            return NotImplemented
        return self._total_value() < other._total_value()

    def __le__(self, other) -> bool:
        if not isinstance(other, Portfolio):
            return NotImplemented
        return self._total_value() <= other._total_value()

In [12]:
aapl = Stock("AAPL", 190.0)
tsla = Stock("TSLA", 250.0)

call = EuropeanPutOption(
    name="CALL_AAPL_185",
    price=12.5,
    expiration_date="2025-12-31",
    underlying_name="AAPL",
    strike_price=185.0,
    risk_free_rate=0.05,
    volatility=0.25
)

portfolio = Portfolio([aapl, tsla, call])

print(portfolio['AAPL'].price)      # 190.0
print('TSLA' in portfolio)          # True
# (190 + 12.5) * 2 = 405.0

190.0
True


**6. ПРОДВИНУТЫЕ ЗАДАЧИ:**
- Создайте декоратор `@validate_option_params` для валидации параметров опционов
- Реализуйте класс `PortfolioManager` с методами для управления несколькими портфелями
- Добавьте поддержку сериализации/десериализации портфелей (pickle)
- Создайте контекстный менеджер для временного изменения параметров портфеля

In [ ]:
import copy
import pickle
from contextlib import contextmanager
from datetime import datetime


def validate_option_params(cls):
    """
        Class decorator to validate option parameters during initialization.

        Validates:
            - strike_price > 0
            - volatility > 0
            - risk_free_rate >= 0 (allows rates > 1 for flexibility)
            - expiration_date is in the future

        Applied to Option subclasses to wrap their __init__ method.
    """
    original_init = cls.__init__

    def new_init(self, name: str, price: float, expiration_date: str, underlying_name: str,
                 strike_price: float, risk_free_rate: float, volatility: float):
        # validate numerical parameters
        if strike_price <= 0:
            raise ValueError("Strike price must be positive")
        if volatility <= 0:
            raise ValueError("Volatility must be positive")
        if risk_free_rate < 0:
            raise ValueError("Risk-free rate cannot be negative")
        # expiration_date validation is already in Derivative

        # call classmethod __init__
        original_init(self, name, price, expiration_date, underlying_name,
                      strike_price, risk_free_rate, volatility)

    cls.__init__ = new_init
    return cls


class PortfolioManager:
    """
        Manager for multiple portfolios.

        Stores portfolios in a dictionary by name.
        Provides safe access, listing, and removal.

        Class attributes:
            _portfolios (dict): registry of portfolios {name: Portfolio}
    """

    _portfolios: dict[str, 'Portfolio'] = {}

    @classmethod
    def add_portfolio(cls, name: str, portfolio: 'Portfolio') -> None:
        # add portfolio under given name; overwrite if exists
        cls._portfolios[name] = portfolio

    @classmethod
    def get_portfolio(cls, name: str) -> 'Portfolio':
        # retrieve portfolio by name; raises KeyError if not found
        if name not in cls._portfolios:
            raise KeyError(f"Portfolio '{name}' not found")
        return cls._portfolios[name]

    @classmethod
    def remove_portfolio(cls, name: str) -> None:
        # remove portfolio by name
        if name not in cls._portfolios:
            raise KeyError(f"Portfolio '{name}' not found")
        del cls._portfolios[name]

    @classmethod
    def list_portfolios(cls) -> list[str]:
        # return list of all portfolio names
        return list(cls._portfolios.keys())

# class TemporaryPriceChange:
#     """
#         Context manager to temporarily change the price of an asset.

#         Restores original price on exit.

#         Args:
#             asset (AbstractAsset): asset to modify
#             new_price (float): temporary price to set
#     """

#     def __init__(self, asset: AbstractAsset, new_price: float):
#         self.asset = asset
#         self.new_price = new_price
#         self.original_price = asset.price

#     def __enter__(self):
#         # apply temporary price
#         self.asset.price = self.new_price
#         return self.asset

#     def __exit__(self, exc_type, exc_val, exc_tb):
#         # restore original price, even if exception occurred
#         self.asset.price = self.original_price

In [ ]:
EuropeanPutOption = validate_option_params(EuropeanPutOption)
AmericanCallOption = validate_option_params(AmericanCallOption)
BinaryOption = validate_option_params(BinaryOption)
BarrierOption = validate_option_params(BarrierOption)

In [ ]:
### Code usage (pickle serialization) 

#   data = pickle.dumps(portfolio)
#   restored = pickle.loads(data)